## Chap04-1. Simple Classification
### 털과 날개가 있는지 없는지에 따라, 포유류인지 조류인지 분류하는 신경망 모델 구현

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
# 1) 학습 데이터 정의

# [털, 날개]
x_data = np.array([[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])

# x_data에 해당하는 labeling 작업
# [기타, 포유류, 조류] one-hot encoding 방식으로 입력
y_data = np.array([
    [1, 0, 0], # 기타
    [0, 1, 0], # 포유류
    [0, 0, 1], # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

In [7]:
# 2) 신경망 모델 구성

# Input(X), Output(Y) 설정
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# Parameter(W, b) 설정
#  - W: [입력층(input), 출력층(output)] -->> [2, 3]으로 설정 
#  - b: 편향(bias)은 ouput의 개수로 설정 
W = tf.Variable(tf.random_uniform([2, 3], -1., 1.))
b = tf.Variable(tf.zeros([3]))

# 신경망에 가중치 W와 편향 b를 적용
L = tf.add(tf.matmul(X, W), b)

# XW + b 한 값에 ReLU 활성화 함수 적용
L = tf.nn.relu(L)

# 분류 문제이므로 ReLU의 출력값에 Softmax를 적용
# model = tf.nn.softmax(L)
model = L

# 손실함수(Loss function): Cross-Entropy를 이용
#  - https://www.tensorflow.org/api_docs/python/tf/losses/softmax_cross_entropy
#  - tf.losses.softmax_cross_entropy를 이용
# cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis=1))
cost = tf.losses.softmax_cross_entropy(onehot_labels=Y, logits=model)

# 최적화 함수 설정: GradientDescent 사용
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

In [22]:
# 3) 신경망 모델 학습

# Tensorflow 세션 초기화
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    if(step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

10 1.09164
20 1.08892
30 1.08628
40 1.08372
50 1.08176
60 1.08098
70 1.08022
80 1.07945
90 1.0787
100 1.07796


In [23]:
# 4) 결과 확인
# 학습데이터가 적어 정확도는 매 실행마다 다르게 나옴

# tf.argmax를 이용해 예측한 결과 중 가장 큰 값을 가져옴
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값: ', sess.run(prediction, feed_dict={X: x_data}))
print('실제값: ', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: {:.2f}'.format(sess.run(accuracy * 100, feed_dict={X: x_data, Y:y_data})))

예측값:  [0 1 1 0 0 0]
실제값:  [0 1 2 0 0 2]
정확도: 66.67
